In [2]:
from tensorflow import keras


model = keras.models.load_model(r"C:\Users\sohan\Downloads\model_checkpoint.h5")

ValueError: Unknown layer: Patches. Please ensure this object is passed to the `custom_objects` argument. See https://www.tensorflow.org/guide/keras/save_and_serialize#registering_the_custom_object for details.

In [ ]:
import tensorflow as tf
from keras import backend as K
import matplotlib.pyplot as plt
import cv2

In [7]:
from PIL import Image
import numpy as np

In [8]:
img_path = "C:\Training\Training\BCC\ISIC_0054121.jpg"

img = Image.open(img_path)
img_resized = img.resize((128,128))
pixels = np.asarray(img_resized)  # convert image to array
pixels = pixels.astype('float32')

input = np.expand_dims(pixels, axis=0)

species = ['bcc','bkl','mel','nv']

preds = model.predict(input)
print(species[np.argmax(preds[0])])

with tf.GradientTape() as tape:
        last_conv_layer = model.get_layer('logits')
        iterate = tf.keras.models.Model([model.inputs], [model.output, last_conv_layer.output])
        model_out, last_conv_layer = iterate(input)
        class_out = model_out[:, np.argmax(model_out[0])]
        grads = tape.gradient(class_out, last_conv_layer)
        pooled_grads = K.mean(grads, axis=(0, 1, 2))

heatmap = tf.reduce_mean(tf.multiply(pooled_grads, last_conv_layer), axis=-1)

heatmap = np.maximum(heatmap, 0)
heatmap /= np.max(heatmap)
heatmap = heatmap.reshape((5, 5))
plt.matshow(heatmap)
plt.show()

INTENSITY = 0.5

raw = Image.open(img_path)
raw = raw.resize((128,128))
heatmap = cv2.resize(heatmap, (128,128))
heatmap = cv2.applyColorMap(np.uint8(255*heatmap), cv2.COLORMAP_JET)


img = (heatmap * INTENSITY + np.array(raw))/255

plt.imshow(raw)
plt.show()
plt.imshow(img)
plt.show()

NameError: name 'model' is not defined